In [1]:
DATA_NAME = 'segmentAnything-full' 
TRANSFORM = 'learned'
CHANNEL = ''

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_learned_full_segmentAnything.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,5.92,0.00,0.00,0.00,-0.00,-0.0,0.0,0.0,0.0,0.0,...,0.0,-0.0,0.0,0.0,-0.0,0.00,-0.00,0.00,0.00,0.00
1,0.00,6.55,-0.00,0.00,-0.00,0.0,-0.0,-0.0,0.0,0.0,...,-0.0,0.0,-0.0,0.0,-0.0,0.00,0.00,0.00,-0.00,-0.00
2,0.00,-0.00,7.25,-0.00,0.00,0.0,0.0,0.0,-0.0,-0.0,...,0.0,-0.0,0.0,-0.0,0.0,-0.00,0.00,-0.00,0.00,0.00
3,0.00,0.00,-0.00,5.09,-0.00,-0.0,-0.0,-0.0,0.0,-0.0,...,0.0,-0.0,0.0,0.0,0.0,-0.00,-0.00,-0.00,0.00,-0.00
4,-0.00,-0.00,0.00,-0.00,9.41,0.0,-0.0,0.0,-0.0,0.0,...,0.0,0.0,-0.0,0.0,0.0,-0.00,-0.00,-0.00,0.00,-0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,0.00,0.00,-0.00,-0.00,-0.00,-0.0,0.0,-0.0,-0.0,-0.0,...,-0.0,-0.0,0.0,0.0,-0.0,7.15,0.00,-0.00,0.00,-0.00
60,-0.00,0.00,0.00,-0.00,-0.00,0.0,0.0,-0.0,0.0,-0.0,...,0.0,0.0,-0.0,0.0,-0.0,0.00,0.76,-0.00,0.00,-0.00
61,0.00,0.00,-0.00,-0.00,-0.00,0.0,-0.0,0.0,0.0,0.0,...,0.0,-0.0,-0.0,-0.0,-0.0,-0.00,-0.00,3.12,0.00,0.00
62,0.00,-0.00,0.00,0.00,0.00,0.0,-0.0,-0.0,-0.0,0.0,...,-0.0,-0.0,0.0,0.0,-0.0,0.00,0.00,0.00,5.92,0.00


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,1.00000,0.00001,0.00001,0.00001,-0.00003,-0.00001,0.00002,0.00002,0.00005,0.00006,...,0.00003,-0.00001,0.00002,0.00002,-0.00004,0.00007,-0.00005,0.00003,0.00001,0.00002
1,0.00001,1.00000,-0.00001,0.00001,-0.00003,0.00002,-0.00001,-0.00000,0.00005,0.00001,...,-0.00000,0.00003,-0.00003,0.00004,-0.00004,0.00004,0.00003,0.00001,-0.00002,-0.00006
2,0.00001,-0.00001,1.00000,-0.00001,0.00003,0.00005,0.00002,0.00002,-0.00006,-0.00000,...,0.00004,-0.00001,0.00002,-0.00002,0.00000,-0.00004,0.00002,-0.00006,0.00003,0.00000
3,0.00001,0.00001,-0.00001,1.00000,-0.00001,-0.00001,-0.00000,-0.00001,0.00002,-0.00002,...,0.00000,-0.00003,0.00003,0.00005,0.00005,-0.00002,-0.00002,-0.00002,0.00002,-0.00001
4,-0.00003,-0.00003,0.00003,-0.00001,1.00000,0.00006,-0.00001,0.00005,-0.00000,0.00002,...,0.00002,0.00001,-0.00006,0.00001,0.00003,-0.00002,-0.00002,-0.00001,0.00003,-0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,0.00007,0.00004,-0.00004,-0.00002,-0.00002,-0.00002,0.00003,-0.00007,-0.00004,-0.00000,...,-0.00001,-0.00005,0.00004,0.00002,-0.00002,1.00000,0.00003,-0.00002,0.00000,-0.00000
60,-0.00005,0.00003,0.00002,-0.00002,-0.00002,0.00002,0.00000,-0.00002,0.00002,-0.00000,...,0.00002,0.00008,-0.00003,0.00004,-0.00003,0.00003,1.00000,-0.00002,0.00001,-0.00000
61,0.00003,0.00001,-0.00006,-0.00002,-0.00001,0.00001,-0.00003,0.00006,0.00007,0.00004,...,0.00000,-0.00003,-0.00006,-0.00002,-0.00005,-0.00002,-0.00002,1.00000,0.00001,0.00001
62,0.00001,-0.00002,0.00003,0.00002,0.00003,0.00001,-0.00000,-0.00005,-0.00001,0.00001,...,-0.00001,-0.00000,0.00001,0.00000,-0.00001,0.00000,0.00001,0.00001,1.00000,0.00003


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.009307979013039782

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[5.93887182e+00 4.85318217e+00 1.51031281e+00 1.45196492e+00
 9.70617019e-01 8.69258209e-01 8.42276322e-01 7.85988818e-01
 7.43188494e-01 7.07649092e-01 6.75125069e-01 6.52058704e-01
 6.36085835e-01 5.92992043e-01 5.87567955e-01 5.84683379e-01
 5.70136586e-01 5.47383164e-01 5.17365165e-01 4.79007253e-01
 4.63774393e-01 4.27763205e-01 4.12094510e-01 3.95502895e-01
 3.51277669e-01 3.44847607e-01 3.38680249e-01 3.24826203e-01
 3.02548032e-01 2.66489748e-01 2.62383438e-01 2.59098951e-01
 2.54785572e-01 2.49286984e-01 2.42488516e-01 2.38999272e-01
 2.34049845e-01 2.28010552e-01 2.16013486e-01 2.02190754e-01
 1.85487207e-01 1.79701558e-01 1.76849556e-01 1.72358030e-01
 1.68178219e-01 1.64350834e-01 1.61326572e-01 1.55987503e-01
 1.50879584e-01 1.46965115e-01 1.45002315e-01 1.29328050e-01
 1.22385542e-01 1.18014272e-01 1.09690431e-01 9.63170794e-02
 8.92668346e-02 4.51799974e-02 1.44719241e-02 4.33668155e-03
 1.13284613e-35]

Principal components (eigenve

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,-0.005002,-0.005666,-0.006412,-0.004123,-0.009439,-0.005019,-0.002544,-0.003143,-0.000273,-0.002128,...,-0.003112,-0.002576,-0.005425,-0.003004,-0.002317,-0.006269,-0.000566,-0.002416,-0.004953,-0.001760
1,-0.006468,-0.007389,-0.008516,-0.005342,-0.013055,-0.006503,-0.003261,-0.004037,-0.000353,-0.002680,...,-0.004011,-0.003275,-0.007153,-0.003813,-0.002983,-0.008319,-0.000735,-0.003091,-0.006472,-0.002240
2,-0.007689,-0.009969,-0.013900,-0.005671,-0.341165,-0.007880,-0.002970,-0.003862,-0.000280,-0.002418,...,-0.003790,-0.003007,-0.009277,-0.003633,-0.002633,-0.013226,-0.000604,-0.002789,-0.007741,-0.001971
3,-0.016953,-0.022033,-0.031011,-0.012376,0.936443,-0.017098,-0.006381,-0.008216,-0.000607,-0.005111,...,-0.008189,-0.006391,-0.020661,-0.007741,-0.005636,-0.029587,-0.001303,-0.005985,-0.016829,-0.004169
4,-0.017864,-0.029667,-0.087176,-0.010866,0.021583,-0.018185,-0.004785,-0.006546,-0.000446,-0.003776,...,-0.006446,-0.004844,-0.025777,-0.006049,-0.004234,-0.069338,-0.000910,-0.004457,-0.017654,-0.002976
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,-0.007683,-0.006662,-0.005907,-0.009474,-0.004326,-0.007533,-0.023241,-0.014577,0.002618,-0.048403,...,-0.014720,-0.023086,-0.006888,-0.016133,-0.032044,-0.006019,0.006068,-0.027435,-0.007714,0.923165
57,-0.024690,-0.022057,-0.019647,-0.029523,-0.014801,-0.024474,-0.055079,-0.040999,0.018626,-0.076755,...,-0.041224,-0.054901,-0.022622,-0.043824,-0.064833,-0.019953,0.047731,-0.060017,-0.024757,-0.121308
58,-0.049653,-0.044753,-0.040262,-0.058242,-0.030757,-0.049206,-0.095143,-0.076321,0.137955,-0.118114,...,-0.076740,-0.094952,-0.045854,-0.080491,-0.106376,-0.040855,0.730463,-0.100999,-0.049674,-0.149853
59,-0.040666,-0.036737,-0.033130,-0.047376,-0.025451,-0.040279,-0.074646,-0.061108,0.639736,-0.090167,...,-0.061379,-0.074480,-0.037634,-0.064129,-0.082403,-0.033601,-0.580279,-0.078748,-0.040647,-0.109633


In [13]:
cos_dist = spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine')
cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.003475677715154446,
 0.0037366278007519016,
 0.06118235003773875,
 0.0635566218176784,
 0.011420057917178705,
 0.05557801642962157,
 0.06875306298964845,
 0.040623820062513194,
 0.04977845731424213,
 0.05810860275686247,
 0.07206525161531019,
 0.1539112500059301,
 0.18664370866625735,
 0.11161455156367983,
 0.20638311725561442,
 0.2717655742237596,
 0.30201463520622274,
 0.35816664990020763,
 0.184199067226538,
 0.2811341935075524,
 0.37188054142592064,
 0.10699062714156882,
 0.21844557059214575,
 0.26488746730716894,
 0.1213085684106745,
 0.21400990406420106,
 0.2902963074466387,
 0.42872023227856315,
 0.11996451471189384,
 0.08449406987390229,
 0.20221882396393864,
 0.3081183431822565,
 0.29936958271555025,
 0.38953995480468273,
 0.22438610123132996,
 0.2835798503482031,
 0.24648937652022007,
 0.18967942845705155,
 0.28609767533505626,
 0.3503530289755208,
 0.1161379108034738,
 0.24459024556344466,
 0.37351173519053626,
 0.3371759692599554,
 0.3183688463002716,
 0.3908357681038213